In [ ]:
%pip install ultralytics
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
print(torch.cuda.is_available())  # True가 나오면 성공

In [ ]:
from ultralytics import YOLO

# 모델 불러오기 (사전 학습된 YOLOv8n, 또는 YOLOv8s/YOLOv8m/YOLOv8l 등으로 변경 가능)
model = YOLO('yolov8n.pt')

# 모델 학습 시작
model.train(
    # ✅ 데이터 경로 및 기본 설정
    data='data.yaml',         # 데이터셋 YAML 파일 경로 (클래스 및 train/val 경로 포함)
    epochs=100,               # 전체 학습 에폭 수
    imgsz=640,                # 입력 이미지 크기 (정사각형 가정)
    batch=16,                 # 배치 사이즈
    device=0,                 # 학습에 사용할 디바이스 번호 또는 'cpu'
    workers=4,                # 데이터 로딩에 사용할 subprocess 개수
    # project='runs/train',     # 결과 저장 루트 디렉토리
    # name='exp_full_v8',       # 하위 폴더 이름
    # exist_ok=True,            # 동일한 폴더가 존재해도 덮어쓰기 허용

    # ✅ 학습률 및 옵티마이저
    lr0=0.01,                 # 초기 학습률
    lrf=0.01,                 # 최종 학습률 (cosine decay의 끝 값)
    momentum=0.937,           # SGD의 관성값
    weight_decay=0.0005,      # L2 정규화 (과적합 방지)
    optimizer='SGD',          # 옵티마이저 (SGD, Adam, AdamW 등 가능)

    # ✅ 스케줄링 및 조기 종료
    patience=50,              # 조기 종료 기준 (성능 개선 없는 에폭 수)
    warmup_epochs=3.0,        # 워밍업 에폭 수
    warmup_momentum=0.8,      # 워밍업 초기 모멘텀
    warmup_bias_lr=0.1,       # 워밍업 시 바이어스 학습률

    # ✅ 증강 (Augmentation) 설정
    # hsv_h=0.015,              # Hue 변화 범위
    # hsv_s=0.7,                # Saturation 변화 범위
    # hsv_v=0.4,                # Value 변화 범위
    # degrees=0.0,              # 회전 각도
    # translate=0.1,            # 이동 범위
    # scale=0.5,                # 스케일 (확대/축소) 비율
    # shear=0.0,                # 기울이기
    # perspective=0.0,          # 원근 왜곡
    # flipud=0.0,               # 상하 반전 확률
    # fliplr=0.5,               # 좌우 반전 확률
    # mosaic=1.0,               # Mosaic 데이터 증강 사용 여부
    # mixup=0.0,                # MixUp 증강 비율
    # copy_paste=0.0,           # Copy-Paste 증강 비율

    # ✅ 정규화 및 기타 손실 제어
    # label_smoothing=0.0,      # 라벨 스무딩 비율
    # dropout=0.0,              # 드롭아웃 비율 (선택사항)

    # ✅ 손실 함수 weight 설정
    # box=7.5,                  # box loss 가중치
    # cls=0.5,                  # class loss 가중치
    # dfl=1.5,                  # Distribution Focal Loss 가중치

    # ✅ 학습 제어
    # val=True,                 # 매 에폭마다 validation 수행 여부
    # save=True,                # 모델 체크포인트 저장 여부
    # save_period=10,           # 몇 에폭마다 저장할지 설정
    # verbose=True,             # 자세한 로그 출력 여부
    # seed=42,                  # 랜덤 시드 고정
    # deterministic=True,       # 재현성 확보를 위한 결정론적 연산

    # ✅ 고급 기능
    # amp=True,                 # Automatic Mixed Precision 사용 여부 (True가 일반적으로 더 빠름)
    # rect=False,               # 사각형 배치 구성 (이미지 비율 유지)
    # resume=False,             # 학습 재시작 여부 (last.pt 필요)
    # pretrained=True,          # 사전 학습된 가중치 사용 여부
    # overlap_mask=True,        # segmentation에서 마스크가 겹칠 때 처리 방식
    # mask_ratio=4,             # segmentation 마스크 축소 비율
)


In [ ]:
# Jupyter Notebook에서 이미지를 셀 내부에 표시하기 위한 설정
%matplotlib inline

import os
import random
import yaml
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2

# --- 설정 ---
VAL_IMG_DIR = 'images/val'
VAL_LABEL_DIR = 'labels/val'
# ===> 바로 여기! 모델 경로를 방금 훈련한 'train4'로 수정했습니다.
MODEL_PATH = 'runs/detect/train4/weights/best.pt' 
DATA_YAML_PATH = 'data.yaml'
CONF_THRESHOLD = 0.25 # 이제 모델 성능이 좋으니 임계값을 0.25 정도로 다시 올려도 됩니다.

# --- 1. 클래스 이름 불러오기 ---
try:
    with open(DATA_YAML_PATH, 'r', encoding='utf-8') as f:
        data = yaml.safe_load(f)
        CLASS_NAMES = data['names']
except Exception as e:
    print(f"'{DATA_YAML_PATH}' 파일을 읽는 중 오류 발생: {e}")
    CLASS_NAMES = []

# --- 2. 비교할 이미지 및 라벨 파일 무작위 선택 ---
image_name = random.choice([f for f in os.listdir(VAL_IMG_DIR) if f.lower().endswith('.jpg')])
img_path = os.path.join(VAL_IMG_DIR, image_name)
label_path = os.path.join(VAL_LABEL_DIR, os.path.splitext(image_name)[0] + '.txt')

# --- 3. 이미지 불러오기 및 모델 예측 ---
image = cv2.imread(img_path)
h, w, _ = image.shape
display_image = image.copy()

# 'train4'에 있는 새로운 모델을 로드합니다.
model = YOLO(MODEL_PATH)
results = model(image, conf=CONF_THRESHOLD, verbose=False)
predictions = results[0]

# --- 4. 실제 정답(Ground Truth) 바운딩 박스 그리기 (초록색) ---
if os.path.exists(label_path) and CLASS_NAMES:
    with open(label_path, 'r') as f:
        for line in f:
            class_id, x_center, y_center, width_norm, height_norm = map(float, line.strip().split())
            x_center_abs, y_center_abs = x_center * w, y_center * h
            width_abs, height_abs = width_norm * w, height_norm * h
            x1, y1 = int(x_center_abs - width_abs / 2), int(y_center_abs - height_abs / 2)
            x2, y2 = int(x_center_abs + width_abs / 2), int(y_center_abs + height_abs / 2)

            cv2.rectangle(display_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label_text = f"{CLASS_NAMES[int(class_id)]} (GT)"
            cv2.putText(display_image, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# --- 5. 모델 예측 바운딩 박스 그리기 (파란색) ---
if CLASS_NAMES and len(predictions.boxes) > 0:
    for box in predictions.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf, cls = box.conf[0], int(box.cls[0])
        
        cv2.rectangle(display_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
        label_text = f"{CLASS_NAMES[cls]} {conf:.2f}"
        cv2.putText(display_image, label_text, (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

# --- 6. 최종 결과 이미지 표시 ---
display_image_rgb = cv2.cvtColor(display_image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(12, 12))
plt.imshow(display_image_rgb)
plt.title(f"Green: Ground Truth  |  Blue: Prediction\nFile: {image_name}")
plt.axis('off')
plt.show()
